<a href="https://colab.research.google.com/github/Uemerson/lstm-predict-stock-prices/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading the Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset_train = pd.read_csv('NSE-TATAGLOBAL.csv')
training_set = dataset_train.iloc[:, 1:2].values

dataset_train.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


#Feature Scaling

In [2]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

#Creating Data with Timesteps

In [3]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#Building the LSTM

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
62/62 [==============================] - 16s 130ms/step - loss: 0.0109
Epoch 2/100
62/62 [==============================] - 8s 130ms/step - loss: 0.0031
Epoch 3/100
62/62 [==============================] - 8s 130ms/step - loss: 0.0033
Epoch 4/100
62/62 [==============================] - 8s 129ms/step - loss: 0.0030
Epoch 5/100
62/62 [==============================] - 8s 129ms/step - loss: 0.0024
Epoch 6/100
62/62 [==============================] - 8s 130ms/step - loss: 0.0022
Epoch 7/100
62/62 [==============================] - 8s 132ms/step - loss: 0.0021
Epoch 8/100
62/62 [==============================] - 8s 132ms/step - loss: 0.0020
Epoch 9/100
62/62 [==============================] - 8s 129ms/step - loss: 0.0021
Epoch 10/100
62/62 [==============================] - 8s 131ms/step - loss: 0.0021
Epoch 11/100
62/62 [==============================] - 8s 129ms/step - loss: 0.0019
Epoch 12/100
62/62 [==============================] - 8s 132ms/step - loss: 0.0019
Epoch 13/100

#Save trained model

In [10]:
import os

regressor.save("model.h5")
print("model.h5 saved model to disk in ", os.getcwd())

model.h5 saved model to disk in  /content
